# Facts and variables

When connecting to an host, ansible creates a dictionary with a huge set of useful informations.

This step is named **fact gathering**. 

You can inspect facts using the `setup` module.

In [1]:
cd exercise-03

In [2]:
# Let's run the setup module

!ansible -i inventory -m setup localhost 

## Exercise

 - run the following command
 - what does it do?
 - where's  its output ?
 - use the json module to reindent it

In [3]:
!ansible >/dev/null -i inventory -m setup localhost --tree host_status

In [4]:
# Solution
!tree host_status
import json
ret = json.load(open('host_status/localhost'))


## Exercise

Gather the following infos from the output of the previous command.

  - hostname
  - first ip address
  - distribution
  - kernel release



In [5]:
# use this cell for the exercise

## Exercise

modify [this playbook](/edit/notebooks/exercise-03/get-facts.yml) to print:


  - hostname
  - first ip address
  - distribution
  - kernel release


In [6]:
!ansible-playbook get-facts.yml

## Facts vs Variables

Essentially, facts are per-host variables gathered by ansible.

In the vault lesson we've seen variables in action loaded it via -e @variables.yml.

Now we'll define them with 

```
- hosts: localhost
  vars:
  - one_variable: 1
  - another_variable: 
      that_is: a_map
```

In [7]:
!cat vars-and-facts.yml

In [8]:
# This playbook mixes variables and facts
!ansible-playbook vars-and-facts.yml --tags variables

### filters

inside {{ braces }} you can process expressions using filters.

```

  - debug:
      msg: >
        Floor it  {{ ( one + pi ) | int }}


```

In [9]:
# This playbook mixes variables and facts
!ansible-playbook vars-and-facts.yml --tags filters

## Exercise

Edit [the last section of vars-and-facts.yml](/edit/notebooks/exercise-03/vars-and-facts.yml) so that it shows:
    
    - the free percentage of every device

Hints:
    
    - use the playbook as a reference
    - iterate thru server facts

In [10]:
!ansible-playbook vars-and-facts.yml --tags exercise

## Conditions and Assertions

Ansible supports basic condition checking via when:
    
```
- hosts: localhost
  tasks:    
  - debug: msg="This always happens"
    when: true
  - debug: msg="This never does"
    when: false
```


You can use `when` with variables and conditions too:

```
- hosts: localhost  
  tasks:    
  - debug: msg="This is {{ansible_kernel}}!"
    when: ansible_system == 'Linux'

```

Remember:

*** when statements should not include jinja2 templating delimiters such as {{ }} or {% %}. ***


In [11]:
!ansible-playbook conditions.yml --tags when

## assertions

The fail module is used to make assertions. Unless told otherwise, ansible playbooks stops when a task fail.

```
- hosts: localhost  
  tags: assert
  tasks:    
  - fail: msg="This is Linux!"
    when: ansible_system != 'Linux'

  - fail: msg="Stop processing now!"
    when: ansible_architecture == 'x86_64'

```

In [12]:
!ansible-playbook conditions.yml --tags assert

## Exercise

Edit [conditions.yml](/edit/notebooks/exercise-03/conditions.yml) and add a check on free disk size > 75%.

  - get informations from gather_facts
  - iterate thru partition `with_items`
  - use `when` and fail module
    

In [13]:
!ansible-playbook conditions.yml --tags exercise

## Predefined variables

There is a set of predefined variables too:

  - group_names
  - groups
  - environment 
  
When using playbooks, we'll get this one too:

  - hostvars


In [19]:
!ansible all[0] -i ../web -m debug -a "var=groups"

### Exercise

Show the content of all 4 variables:

  - is every variable defined?
  - is every variable not empty?

In [1]:
# Use this cell for the exercise

## Shell environment & Files

Ansible can iterate thru *local* files